In [1]:
%config IPCompleter.greedy=True
from collections import Counter

In [8]:
from flask_sqlalchemy import SQLAlchemy
from flask import Flask
from datetime import datetime, timedelta, date, time
import random
import calendar

# from sqlalchemy.orm import Session
from sqlalchemy import select

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///:memory:'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)


class Task(db.Model):
    """Data Model for Events Happening"""

    __tablename__ = 'tasks'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(64), index=False, unique=False, nullable=False)
    category = db.Column(db.String(64), index=False, unique=False, nullable=True)
    subcategory1 = db.Column(db.String(64), index=False, unique=False, nullable=True)
    subcategory2 = db.Column(db.String(64), index=False, unique=False, nullable=True)
    priority = db.Column(db.Integer, index=False, unique=False, nullable=False)
    add_date = db.Column(db.DateTime, index=False, unique=False, nullable=False)
    complete_date = db.Column(db.DateTime, index=False, unique=False, nullable=True)

    def __repr__(self):
        return f'''Task(name = {self.name}, category = {self.category},
            subcategory1 = {self.subcategory1}, subcategory2 = {self.subcategory2},
            priority = {self.priority}, 
            add_date = {self.add_date}, complete_date = {self.complete_date}
            '''


db.init_app(app)
db.create_all()


def add_fake_tasks(number):
    for _ in range(number):
        tstamp = datetime.utcnow() - timedelta(days=random.randint(0, 100))
        completed = (tstamp + timedelta(days=random.randint(0, 100)), None, None, None)
        task = {
            'name': f'task {random.random()}',
            'category': 'financial',
            'subcategory1': None,
            'subcategory2': None,
            'priority': random.randint(0, 100),
            'add_date': tstamp,
            'complete_date': random.choice(completed),
        }
        db.session.add(Task(**task))
    db.session.commit()


add_fake_tasks(500)


In [9]:
today = datetime.combine(date.today(), time())
tomorrow = today + timedelta(days=1)
yesterday = today - timedelta(days=1)
previous_7 = today - timedelta(days=7)
previous_30 = today - timedelta(days=30)
_month_days = calendar.monthrange(today.year, today.month)[1]
week_number = today.isocalendar().week
week_start = datetime.combine(date.fromisocalendar(today.year, week_number, 1), time())
week_end = week_start + timedelta(days=7)
this_month = datetime(today.year, today.month, 1)
next_month = this_month + timedelta(days=_month_days)
this_year = datetime(today.year, 1, 1)
next_year = datetime(today.year + 1, 1, 1)


In [10]:
completed = db.session.execute(
    select(Task)
    .where(Task.complete_date > this_month, Task.complete_date < next_month)
    .order_by(Task.priority.asc(), Task.add_date.asc())
).scalars().all()

In [11]:
def datetimes_between_two_dates(start: datetime, end: datetime) -> datetime:
    return [start + timedelta(days=x) for x in range((end - start).days)]

In [17]:
dts = {dt: 0 for dt in datetimes_between_two_dates(this_month, next_month)}

count = Counter(
    datetime(
        task.complete_date.year, task.complete_date.month, task.complete_date.day
    )
    for task in reversed(completed)
)
updated = {**dts, **count}

In [ ]:
Counter(
    datetime(
        task.complete_date.year, task.complete_date.month, task.complete_date.day
    )
    for task in reversed(completed)
)

In [36]:
com = completed[0].complete_date
com

datetime.datetime(2022, 4, 10, 7, 44, 47, 256726)

In [42]:
time()

datetime.time(0, 0)

In [40]:
date.fromtimestamp()

TypeError: 'datetime.datetime' object cannot be interpreted as an integer

In [49]:
labels = [datetime.strftime(dt, '%m/%d') for dt in updated]
[int(label.replace('/', '')) for label in labels]

[401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 411,
 412,
 413,
 414,
 415,
 416,
 417,
 418,
 419,
 420,
 421,
 422,
 423,
 424,
 425,
 426,
 427,
 428,
 429,
 430]

In [20]:
datetime(2000, 1, 1)

datetime.datetime(2000, 1, 1, 0, 0)

In [29]:
first_completed_task = (
        db.session.execute(select(Task).where(Task.complete_date.is_not(None)).order_by(Task.complete_date.asc()))
        .scalars()
        .first()
    )
(tomorrow - first_completed_task.complete_date).days

94

In [31]:
first_completed_task.complete_date

datetime.datetime(2022, 1, 5, 7, 44, 47, 254913)